In [1]:
import pexpect
import time
import sys
import re
import datetime

In [2]:
#ENM3 -> 10.23.202.28 PW : "Enm#2021"
#ENM5 -> 10.167.30.28 PW : "Enm#2021"

child = pexpect.spawn('ssh -L 7898:localhost:8007 -J root@150.236.203.198 root@127.0.0.1 -p 8007', timeout=30, encoding='utf-8', logfile=sys.stdout)

child.maxread = 300000 

child.expect("root@150.236.203.198's password: ")
child.sendline('PMT@2022')
child.expect("root@127.0.0.1's password: ")
child.sendline('PMT@2022')

time.sleep(3)

# Shell command to take after the dates in modification dates
# child.sendline('find /var/opt/common5/mdt/ -type f -newermt 2023-04-28 -exec ls -l {} \;')

# Shell command to take only the folder
# child.sendline('find /var/opt/common5/mdt/20230205 -type f -exec ls -l {} \;')

# Shell command to take certain range dates in modification dates
child.sendline('find /var/opt/common5/mdt -type f -newermt 2023-05-01 ! -newermt 2023-06-06 -exec ls -l {} \;')

try:
    child.expect(pexpect.EOF, timeout=150)

except pexpect.TIMEOUT:
    child.close()
    raw = str(child.before)

root@150.236.203.198's password: PMT@2022

root@127.0.0.1's password: PMT@2022
find /var/opt/common5/mdt -type f -newermt 2023-05-01 ! -newermt 2023-06-06 -exec ls -l {} \;

Last login: Wed Jun  7 10:01:34 2023 from 192.168.21.6
! -newermt 2023-06-06 -exec ls -l {} \;-1 ~]# find /var/opt/common5/mdt -type f -newermt 2023-05-01  
-rw-r--r--. 1 root root 13157456 May  8 21:25 /var/opt/common5/mdt/20230110/filter_mdt_result.zip
-rw-r--r--. 1 root root 13621999 May  8 21:26 /var/opt/common5/mdt/20230110/filter_ue_traffic.zip
-rw-r--r--. 1 root root 27216465 May  8 21:28 /var/opt/common5/mdt/20230110/filter_ue_meas.zip
-rw-r--r--. 1 root root 12903094 May  8 21:31 /var/opt/common5/mdt/20230111/filter_mdt_result.zip
-rw-r--r--. 1 root root 13096546 May  8 21:33 /var/opt/common5/mdt/20230111/filter_ue_traffic.zip
-rw-r--r--. 1 root root 26501574 May  8 21:37 /var/opt/common5/mdt/20230111/filter_ue_meas.zip
-rw-r--r--. 1 root root 11860048 May  8 21:11 /var/opt/common5/mdt/20230114/filter_mdt_

In [ ]:
test1 = re.split(r'\s+|\n', raw)

In [ ]:
# Create list comprehension to filter list of zip files only

# result = [file for file in test1 if file.endswith('.zip')]

# Create list comprehension to filter only mdt_result.zip only 
# Option are ue_traffic or mdt_result

pattern = r'.*/(?!filter_)[^/]+/ue_traffic\.zip$'
result = [path for path in test1 if re.match(pattern, path)]

In [ ]:
# Create list comprehension to generate weekend dates in May

weekend_dates = [date.strftime("%Y%m%d") for date in (datetime.date(2023, 5, day) for day in range(1, 32)) 
                 if date.weekday() >= 5]

In [ ]:
# Create list comprehension to filter May Month only
result = [path for path in result if '202305' in path]

# Create list comprehension to filter Out Some Date
result = [path for path in result if '20230501' not in path and '20230518' not in path]

# Create list comprehension to filter Out Some Date
# result = [path for path in result if '20230501' not in path and '20230518' not in path]

# Create list comprehension to filter Out Weekend Date
result = [path for path in result if path.split('/')[5] not in weekend_dates]

# Create list comprehension to filter Busy Hour only
# result = [path for path in result if '/07/' in path or '/08/' in path or '/09/' in path or '/17/' in path or '/18/' in path or '/19/' in path]

# Create list comprehension to filter Busy Hour only
result = [path for path in result if '/11/' in path or '/12/' in path or '/13/' in path]

# result = [path for path in result if '/07/' in path or '/08/' in path or '/09/']

In [ ]:
with open(r'/home/nivag/location_to_dl_8007-throughput.txt', 'w') as fp:
    fp.write('\n'.join(result))